# Detection Model Training and Testing Notebook

This notebook demonstrates how to train a simple detection model using PyTorch and then test it on new data. The notebook is divided into several sections: 

1. **Imports and Data Preparation**: Loading the dataset and creating feature representations for each image.
2. **Dataset and Dataloader**: Creating a custom dataset class to handle our data.
3. **Model Definition and Training**: Defining the neural network model, training it, and printing the loss per epoch.
4. **Testing the Model**: Loading new data, running the model in evaluation mode, and printing out predictions or test loss.


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Load training data
df = pd.read_csv(
    "labels_test_8_bit.txt",
    sep=" ",
    names=["image_file", "class_id", "x_min", "x_max", "y_min", "y_max"]
)

# Create a features dictionary for each unique image
features_dict = {}
for img in df['image_file'].unique():
    features_dict[img] = torch.randn(256)  # Replace with your actual feature extraction if available

In [ ]:
# Define a custom Dataset for detection tasks
class DetectionDataset(Dataset):
    def __init__(self, df, features_dict):
        self.df = df
        self.features_dict = features_dict

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_file = row['image_file']
        features = self.features_dict[image_file]
        bbox = torch.tensor([row['x_min'], row['x_max'], row['y_min'], row['y_max']], dtype=torch.float)
        label = torch.tensor(row['class_id'] - 1, dtype=torch.long)
        return features, bbox, label

In [ ]:
# Create the dataset and dataloader for training
dataset = DetectionDataset(df, features_dict)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# Define the baseline detection model
class BaselineDetector(nn.Module):
    def __init__(self, input_dim=256):
        super(BaselineDetector, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU()
        )
        self.bbox_head = nn.Linear(128, 4)
        self.class_head = nn.Linear(128, 3)  # Assuming 3 classes

    def forward(self, x):
        features = self.fc(x)
        bbox = self.bbox_head(features)
        class_logits = self.class_head(features)
        return bbox, class_logits

In [ ]:
# Instantiate the model, define optimizer and loss functions
model = BaselineDetector(input_dim=256)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion_bbox = nn.MSELoss()
criterion_cls = nn.CrossEntropyLoss()

In [ ]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0.0
    for features, bbox, label in dataloader:
        optimizer.zero_grad()
        pred_bbox, pred_class = model(features)
        loss_bbox = criterion_bbox(pred_bbox, bbox)
        loss_cls = criterion_cls(pred_class, label)
        loss = loss_bbox + loss_cls
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")

## Testing the Model on New Data

The following cells show how to test the model on new data. If labels are available in the new data, you can compute the test loss; otherwise, you can simply retrieve the model predictions.

In [ ]:
# Load new data for testing
new_df = pd.read_csv(
    "labels_new_data.txt",  # Replace with your new data file
    sep=" ",
    names=["image_file", "class_id", "x_min", "x_max", "y_min", "y_max"]
)

# Create a new features dictionary for the new images
new_features_dict = {}
for img in new_df['image_file'].unique():
    new_features_dict[img] = torch.randn(256)  # Replace with your actual feature extraction

# Create the test dataset and dataloader
test_dataset = DetectionDataset(new_df, new_features_dict)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Option 1: Compute the test loss if labels are available
model.eval()
test_loss = 0.0
with torch.no_grad():
    for features, bbox, label in test_loader:
        pred_bbox, pred_class = model(features)
        loss_bbox = criterion_bbox(pred_bbox, bbox)
        loss_cls = criterion_cls(pred_class, label)
        loss = loss_bbox + loss_cls
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
print(f"Test Loss: {avg_test_loss:.4f}")

In [ ]:
# Option 2: Get predictions if labels are not available
predictions = []
with torch.no_grad():
    for features, _, _ in test_loader:
        pred_bbox, pred_class = model(features)
        predicted_labels = pred_class.argmax(dim=1)
        predictions.append((pred_bbox, predicted_labels))

if predictions:
    sample_pred_bbox, sample_pred_label = predictions[0]
    print("Sample Predictions:", sample_pred_bbox, sample_pred_label)
else:
    print("No predictions available.")

In [ ]:
# Single sample testing
model.eval()

# Select a single image from the new dataset (here we choose the first one)
sample_image = new_df['image_file'].iloc[0]
sample_features = new_features_dict[sample_image].unsqueeze(0)  # Add batch dimension

with torch.no_grad():
    pred_bbox, pred_class = model(sample_features)
    predicted_label = pred_class.argmax(dim=1)

print("Predicted Bounding Box:", pred_bbox)
print("Predicted Class:", predicted_label)